## Output comparison for different retriever and a set of questions

In [5]:
from harvai.qa_model import get_answer
from harvai.data import get_clean_preproc_data
from harvai.nn_model import Nn_model
from harvai.bm25 import Bm25
from harvai.dpr import DPR
from harvai.embedding import Embedding
from transformers import pipeline
import warnings
import numpy as np


INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [6]:
import pandas as pd 

In [7]:

data = pd.read_csv('../raw_data/generated_questions_dataset.csv')
data.drop(columns=['Unnamed: 0.1','Unnamed: 0'], inplace=True)

In [8]:
data['questions_preproc'].sample(2)

2518             quel mode pose plaque dire plaque exploitation
129     disposition législatif présent livre applicable mayotte
Name: questions_preproc, dtype: object

In [9]:
## retrievers
article_number = 5
retriever_dictonnary =  {"KNN" : Nn_model(article_number), "BM25":Bm25(article_number), "DPR":DPR(article_number), "Embedding":Embedding(article_number)}

## retrievers fitting

for model in retriever_dictonnary.keys():
    retriever_dictonnary[model].clean_data()
    retriever_dictonnary[model].fit()

## reader fitting

reader = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at ../raw_data/retriever.pt/query_encoder
WARNING - haystack.modeling.model.language_model -  Using a model of type 'camembert' which might be incompatible with DPR encoders.Bert based encoders are supported that need input_ids,token_type_ids,attention_mask as input tensors.
INFO - haystack.modeling.model.language_model -  Loaded ../raw_data/retriever.pt/query_encoder
The tokenizer class

In [10]:
data.loc[2781]

questions            Comment le ministre de l'intérieur informe-t-il la réimmatriculation du véhi...
id                                                                                               688
questions_preproc                      comment ministre intérieur informe réimmatriculation véhicule
Name: 2781, dtype: object

In [14]:
questions = data.loc[selected_questions]

In [17]:
## retrievers predicting

#questions  = data.sample(1)

for index,row in questions.iterrows():

    for model in retriever_dictonnary.keys():
        retriever_dictonnary[model].predict(row['questions_preproc'])


    ## display output predicting
    warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
    
    print(f'\n')
    print(f'{row["questions"]}')
    print(f'\n')

    for model in retriever_dictonnary.keys():
        print(f'\n')
        print(f'--------------{model}-------------------------')
        print(f'\n')
        context = retriever_dictonnary[model].get_articles_text_only()
        print(reader({ 'question': row['questions_preproc'], 'context': context }))
        
        
        print(retriever_dictonnary[model].articles)
        
        if(row['id'] in retriever_dictonnary[model].articles):
            print('Retriever succeeded')
            print(row['id'])
        else:
            print('Retriever failed')
            print(row['id'])
            
    print(articles.loc[row['id']]['article_content'])
        
        
        

    






Qui peut recueillir des informations sur le nombre de points détenus par le titulaire d un permis de conduire?




--------------KNN-------------------------


{'score': 0.23561227321624756, 'start': 95, 'end': 201, 'answer': ' ne peuvent être collectées que par les autorités administratives et judiciaires qui doivent en connaître,'}
[79, 80, 432, 430, 73]
Retriever succeeded
79


--------------BM25-------------------------


{'score': 0.2882596552371979, 'start': 95, 'end': 201, 'answer': ' ne peuvent être collectées que par les autorités administratives et judiciaires qui doivent en connaître,'}
[79, 432, 430, 80, 73]
Retriever succeeded
79


--------------DPR-------------------------


{'score': 0.15593481063842773, 'start': 6610, 'end': 6690, 'answer': ' ne peuvent être collectées que par les autorités administratives et judiciaires'}
[82, 83, 459, 79, 100]
Retriever succeeded
79


--------------Embedding-------------------------


{'score': 0.054347433149814606, 'start': 279, 'e

## test de tri du dataset de questions à l'aide du modèle reader

In [16]:
articles = get_clean_preproc_data()

In [319]:
good_questions_ids = []
for index,row in data.sample(10).iterrows(): 
    #output = reader({ 'question': row['questions'], 'context': articles.loc[row['id']]['article_content'] })
    #if output['score'] > 0.9:
    good_questions_ids.append(index)
        

In [320]:
data.loc[good_questions_ids]['questions']

756     En cas d'émission du titre exécutoire prévu à l article L. 2333-87 du code d...
2362    Les règles techniques prévues à la présente section ne sont applicables qu'a...
2648                          Quelle est la vitesse maximale maximale des cyclomoteurs?
510                                               Quel est le second contrôle effectué?
4092    Que doivent être les feux rouges sur une remorque non accouplée à un arrêt o...
2843                Quelle est la durée maximale de l'agrément d'un réseau de contrôle?
2721                               Qu'est-ce qui détermine les catégories de véhicules?
3285    Que peut décider le ministre chargé des transports en cas d'impossibilité de...
325     Qui peut recueillir des informations sur le nombre de points détenus par le ...
1341                                       Que peuvent constater les gardes champêtres?
Name: questions, dtype: object

In [321]:
data.loc[good_questions_ids]['questions'].values

array(["En cas d'émission du titre exécutoire prévu à l article L. 2333-87 du code de procédure pénale, qu'est-ce",
       "Les règles techniques prévues à la présente section ne sont applicables qu'aux véhicules spéciaux des armées si elles sont compatibles avec leurs caractéristiques techniques de fabrication et",
       'Quelle est la vitesse maximale maximale des cyclomoteurs?',
       'Quel est le second contrôle effectué?',
       'Que doivent être les feux rouges sur une remorque non accouplée à un arrêt ou en stationnement sur la chaussée?',
       "Quelle est la durée maximale de l'agrément d'un réseau de contrôle?",
       "Qu'est-ce qui détermine les catégories de véhicules?",
       "Que peut décider le ministre chargé des transports en cas d'impossibilité de présenter ses observations?",
       'Qui peut recueillir des informations sur le nombre de points détenus par le titulaire d un permis de conduire?',
       'Que peuvent constater les gardes champêtres?'], dtype=objec

In [13]:
selected_questions = [325,2721,2648,2479,2822,3636,1657,3891,351,529,4168,215,2301,2308,4385,715,685,741,669,2415,4401,4189,2781,302,3898,4186,2513,3771,2796,3128,2202,2185,4006,2515,165,3622,663,3792,3019,1660,2309,3945,3576,2168,3840,1890,2250,2945,161,2396,167,]
len(selected_questions)

51

In [325]:
data.loc[selected_questions].to_csv('../raw_data/clean_questions_dataset.csv')

In [ ]:

data = pd.read_csv('../raw_data/generated_questions_dataset.csv')

In [326]:
from harvai.utils import score

In [328]:
score(retriever_dictonnary["KNN"])

2267it [00:04, 472.60it/s]                          


{'recall': 0.7547419497132775, 'average rank': 1.6212741087083578}

In [ ]:
score(retriever_dictonnary["KNN"])